## Database Implementation and Data Querying

In [70]:
import sqlite3
import pandas as pd
from pathlib import Path

data_dir = Path("./Dataset")         
db_path = Path("./Database/flight_data.db") 
flights_csv  = data_dir / "flights.csv"     
airports_csv = data_dir / "airports.csv"
airlines_csv = data_dir / "airlines.csv"


In [71]:
airports_df  = pd.read_csv(airports_csv)
airlines_df  = pd.read_csv(airlines_csv)

airports_df = airports_df[["IATA_CODE", "AIRPORT", "CITY", "A_STATE",
                           "COUNTRY", "LATITUDE", "LONGITUDE"]].drop_duplicates()
airlines_df = airlines_df[["IATA_CODE", "AIRLINE"]].drop_duplicates()


In [72]:
flights_df = pd.read_csv(flights_csv)

flights_df = flights_df[["A_YEAR", "A_MONTH","A_DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER",
                         "TAIL_NUMBER", "ORIGIN_AIRPORT",
                         "DESTINATION_AIRPORT","SCHEDULED_DEPARTURE", "DEPARTURE_TIME", "DEPARTURE_DELAY", "TAXI_OUT",
                            "WHEELS_OFF", "SCHEDULED_TIME", "ELAPSED_TIME", "AIR_TIME", "DISTANCE", "WHEELS_ON", "TAXI_IN",
                         "SCHEDULED_ARRIVAL", "ARRIVAL_TIME", "ARRIVAL_DELAY", "DIVERTED", "CANCELLED",
                         "CANCELLATION_REASON", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY",
                         "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"]].drop_duplicates()
flights_df.insert(0, "flight_id",  range(1, len(flights_df)+1))

In [73]:
con = sqlite3.connect(db_path)
con.execute("PRAGMA foreign_keys = ON;")
cur = con.cursor()

for tbl in ("flights", "airports", "airlines"):
    cur.executescript(f"DROP TABLE IF EXISTS {tbl};")

cur.executescript("""
CREATE TABLE airports(
    IATA_CODE TEXT PRIMARY KEY,
    AIRPORT TEXT,
    CITY TEXT,
    A_STATE TEXT,
    COUNTRY TEXT,
    LATITUDE REAL,
    LONGITUDE REAL
);
""")

cur.executescript("""
CREATE TABLE airlines(
    IATA_CODE    TEXT PRIMARY KEY,
    AIRLINE      TEXT
);
""")

cur.executescript("""
CREATE TABLE flights(
    flight_id INT PRIMARY KEY,
    A_YEAR INT,
    A_MONTH INT,
    A_DAY INT,
    DAY_OF_WEEK INT,
    AIRLINE TEXT,
    FLIGHT_NUMBER INT,
    TAIL_NUMBER TEXT,
    ORIGIN_AIRPORT TEXT,
    DESTINATION_AIRPORT TEXT,
    SCHEDULED_DEPARTURE INT,
    DEPARTURE_TIME INT,
    DEPARTURE_DELAY INT,
    TAXI_OUT INT,
    WHEELS_OFF INT,
    SCHEDULED_TIME INT,
    ELAPSED_TIME INT,
    AIR_TIME INT,
    DISTANCE INT,
    WHEELS_ON INT,
    TAXI_IN INT,
    SCHEDULED_ARRIVAL INT,
    ARRIVAL_TIME INT,
    ARRIVAL_DELAY INT,
    DIVERTED INT,
    CANCELLED INT,
    CANCELLATION_REASON TEXT,
    AIR_SYSTEM_DELAY INT,
    SECURITY_DELAY INT,
    AIRLINE_DELAY INT,
    LATE_AIRCRAFT_DELAY INT,
    WEATHER_DELAY INT,
                  
    FOREIGN KEY (ORIGIN_AIRPORT) REFERENCES airports(IATA_CODE),
    FOREIGN KEY (DESTINATION_AIRPORT) REFERENCES airports(IATA_CODE),
    FOREIGN KEY (AIRLINE) REFERENCES airlines(IATA_CODE)
);
""")
con.commit()


In [74]:
airports_df.to_sql("airports", con, if_exists="append", index=False)
airlines_df.to_sql("airlines", con, if_exists="append", index=False)
flights_df.to_sql("flights",  con, if_exists="append", index=False)

con.commit()
con.close()
print("✅ SQLite database built at", db_path)


✅ SQLite database built at Database/flight_data.db


In [75]:
flights_df

,flight_id,A_YEAR,A_MONTH,A_DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,1,2015,1,1,4,AS,98,N407AS,ANC,SEA,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015,1,1,4,US,840,N171US,SFO,CLT,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015,1,1,4,AS,135,N527AS,SEA,ANC,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499996,2015,2,3,2,WN,740,N647SW,DAL,MCO,...,910.0,-20.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
499996,499997,2015,2,3,2,WN,1711,N8315C,MDW,DEN,...,744.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
499997,499998,2015,2,3,2,WN,371,N438WN,MDW,PIT,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
499998,499999,2015,2,3,2,WN,1267,N211WN,MDW,TPA,...,927.0,-18.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
import sqlite3, pandas as pd
from tabulate import tabulate
def run_sql(
    sql: str,
    params: tuple | None = None,
    head_rows: int = 6,       
    tail_rows: int = 3,        
    tablefmt: str = "psql",      
    return_df: bool = False,
):
    with sqlite3.connect(db_path) as con:
        con.execute("PRAGMA foreign_keys = ON;")
        df = pd.read_sql_query(sql, con, params=params)

    n = len(df)
    if n == 0:
        print("(0 rows)")
        return pd.DataFrame() if return_df else None

    if n <= 20:
        short = df
    else:
        ellipsis_row = pd.DataFrame(
            {c: "…" for c in df.columns}, index=["…"]
        )
        short = pd.concat([df.head(head_rows), ellipsis_row, df.tail(tail_rows)])
    in_nb = False
    try:
        from IPython import get_ipython
        in_nb = "IPKernelApp" in get_ipython().config
    except Exception:
        pass

    if in_nb:
        from IPython.display import display, HTML
        display(short)                      
        display(HTML(f"<em>{n:,} rows total</em>"))
    else:
        print(tabulate(short, headers="keys", tablefmt=tablefmt, showindex=False))
        print(f"\n[{n:,} rows total]")

    return df if return_df else None


In [77]:
run_sql("""
SELECT *
FROM airports
""")


,IATA_CODE,AIRPORT,CITY,A_STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
5,ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018
…,…,…,…,…,…,…,…
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023
321,YUM,Yuma International Airport,Yuma,AZ,USA,32.65658,-114.60597


In [78]:
run_sql("""
SELECT *
FROM airlines
""")


,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [79]:
run_sql("""
SELECT *
FROM flights
""")

,flight_id,A_YEAR,A_MONTH,A_DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,1,2015,1,1,4,AS,98,N407AS,ANC,SEA,...,408.0,-22.0,0,0,None,NaN,NaN,NaN,NaN,NaN
1,2,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,...,741.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
2,3,2015,1,1,4,US,840,N171US,SFO,CLT,...,811.0,5.0,0,0,None,NaN,NaN,NaN,NaN,NaN
3,4,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,...,756.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
4,5,2015,1,1,4,AS,135,N527AS,SEA,ANC,...,259.0,-21.0,0,0,None,NaN,NaN,NaN,NaN,NaN
5,6,2015,1,1,4,DL,806,N3730B,SFO,MSP,...,610.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN
…,…,…,…,…,…,…,…,…,…,…,...,…,…,…,…,…,…,…,…,…,…
499997,499998,2015,2,3,2,WN,371,N438WN,MDW,PIT,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
499998,499999,2015,2,3,2,WN,1267,N211WN,MDW,TPA,...,927.0,-18.0,0,0,None,NaN,NaN,NaN,NaN,NaN
499999,500000,2015,2,3,2,WN,136,N352SW,MSP,MKE,...,702.0,-23.0,0,0,None,NaN,NaN,NaN,NaN,NaN


In [80]:
run_sql("""
SELECT F.AIRLINE, COUNT(*) AS NumOfFlights
FROM flights F
GROUP BY F.AIRLINE;
""")


,AIRLINE,NumOfFlights
0,AA,46950
1,AS,14149
2,B6,23062
3,DL,68555
4,EV,52965
5,F9,7291
6,HA,6858
7,MQ,31896
8,NK,9324
9,OO,51184


In [81]:
run_sql("""
SELECT F.AIRLINE, AVG(F.DEPARTURE_DELAY) AS Delay
FROM flights F
GROUP BY F.AIRLINE;
""")


,AIRLINE,Delay
0,AA,10.643729
1,AS,3.430815
2,B6,10.774558
3,DL,6.775430
4,EV,10.043026
5,F9,19.664576
6,HA,1.109583
7,MQ,16.141276
8,NK,14.016792
9,OO,12.458124


In [82]:
run_sql("""
SELECT F.AIRLINE, SUM(F.CANCELLED) AS Cancelled
FROM flights F
GROUP BY F.AIRLINE;
""")

,AIRLINE,Cancelled
0,AA,1324
1,AS,83
2,B6,1479
3,DL,938
4,EV,2523
5,F9,122
6,HA,27
7,MQ,3136
8,NK,158
9,OO,1623


In [83]:
run_sql("""
SELECT F.ORIGIN_AIRPORT, F.DESTINATION_AIRPORT, AVG(F.AIR_TIME) AS AVG_FLY_TIME
FROM flights F
GROUP BY F.ORIGIN_AIRPORT, F.DESTINATION_AIRPORT;
""")

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,AVG_FLY_TIME
0,ABE,ATL,111.540541
1,ABE,DTW,83.267606
2,ABE,ORD,112.263158
3,ABI,DFW,32.394191
4,ABQ,ATL,150.263158
5,ABQ,BWI,192.121212
…,…,…,…
4177,YAK,CDV,36.84375
4178,YAK,JNU,35.133333
4179,YUM,PHX,32.188571


In [84]:
run_sql("""
SELECT f1.flight_id AS FLIGHT_ID, f1.ORIGIN_AIRPORT, a1.A_STATE AS ORIGIN_AIRPORT_STATE, a1.AIRPORT AS ORIGIN_AIRPORT_FULL_NAME,
 f1.DESTINATION_AIRPORT, a2.A_STATE as DESTINATION_AIRPORT_STATE, a2.AIRPORT AS DESTINATION_AIRPORT_FULL_NAME
FROM flights f1, airports a1, airports a2
WHERE f1.ORIGIN_AIRPORT = a1.IATA_CODE AND f1.DESTINATION_AIRPORT = a2.IATA_CODE;
""")

,FLIGHT_ID,ORIGIN_AIRPORT,ORIGIN_AIRPORT_STATE,ORIGIN_AIRPORT_FULL_NAME,DESTINATION_AIRPORT,DESTINATION_AIRPORT_STATE,DESTINATION_AIRPORT_FULL_NAME
0,1,ANC,AK,Ted Stevens Anchorage International Airport,SEA,WA,Seattle-Tacoma International Airport
1,2,LAX,CA,Los Angeles International Airport,PBI,FL,Palm Beach International Airport
2,3,SFO,CA,San Francisco International Airport,CLT,NC,Charlotte Douglas International Airport
3,4,LAX,CA,Los Angeles International Airport,MIA,FL,Miami International Airport
4,5,SEA,WA,Seattle-Tacoma International Airport,ANC,AK,Ted Stevens Anchorage International Airport
5,6,SFO,CA,San Francisco International Airport,MSP,MN,Minneapolis-Saint Paul International Airport
…,…,…,…,…,…,…,…
499997,499998,MDW,IL,Chicago Midway International Airport,PIT,PA,Pittsburgh International Airport
499998,499999,MDW,IL,Chicago Midway International Airport,TPA,FL,Tampa International Airport
499999,500000,MSP,MN,Minneapolis-Saint Paul International Airport,MKE,WI,General Mitchell International Airport


In [85]:
run_sql("""
SELECT f.flight_id AS FLIGHT_ID, f.AIRLINE AS IATA_CODE, a.AIRLINE AS AIRLINE_NAME
FROM flights f, airlines a
WHERE f.AIRLINE = a.IATA_CODE;
""")

,FLIGHT_ID,IATA_CODE,AIRLINE_NAME
0,1,AS,Alaska Airlines Inc.
1,2,AA,American Airlines Inc.
2,3,US,US Airways Inc.
3,4,AA,American Airlines Inc.
4,5,AS,Alaska Airlines Inc.
5,6,DL,Delta Air Lines Inc.
…,…,…,…
499997,499998,WN,Southwest Airlines Co.
499998,499999,WN,Southwest Airlines Co.
499999,500000,WN,Southwest Airlines Co.


## Feature Engineering, Data Preprocessing, and Preparation for Modeling

In [ ]:
# Az inja start bezan. Ghablia ro run nakon chon database ro sakhtab va inja ham df hash ro load kardam az db
con = sqlite3.connect(db_path)
flights_df = pd.read_sql_query("SELECT * FROM flights", con)
airports_df = pd.read_sql_query("SELECT * FROM airports", con)
airlines_df = pd.read_sql_query("SELECT * FROM airlines", con)